# Read clinical-trials covid dataset to set "truth"
 match to drugs in DrugBank based on Name <br>
 requires parsing out and cleansing drug-name from the <font color=blue><b>Interventions</b></font> column

### Additional Steps - for future consideration
Consider parsing <font color=blue>Title</font> field to get phase of clinical trial ?<br>
Consider using <font color=blue>URL</font> column to link to FDA web data<br>
<br>
Consider keeping <font color=blue>Status</font> and <font color=blue>Study Results</font>
<br>
See section below for including an external manual re-mapping of drugs

In [1]:
#import other packages
import pandas as pd
import numpy as np
import os
#from itertools import chain
import time
import getpass


In [2]:
print(os.getcwd())
FDAPath = os.getcwd() + '/../data/source/'
CTPath = FDAPath + 'ClinicalTrials/'
DBPath = FDAPath + "DrugBank/"

os.chdir(CTPath)
print(os.getcwd())


/home/sasdemo05/Python
/home/sasdemo05/data/source/ClinicalTrials


In [3]:
# create process to explode columns with multiple delimited values
#  takes an array column with ['drug: drug1', 'drug: drug2', 'other: ']
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [4]:
display("reading: " + CTPath + 'clinicaltrial_covid.csv')
ctrials_df = pd.read_csv(CTPath + 'clinicaltrial_covid.csv')
ctrials_df.dropna(subset=['Interventions'], inplace=True) # if interventions column is blank.  drop row
display(ctrials_df.head())


'reading: /home/sasdemo05/Python/../data/source/ClinicalTrials/clinicaltrial_covid.csv'

,Rank,Title,Status,Study Results,Conditions,Interventions,Locations,URL
0,1,Duvelisib to Combat COVID-19,Recruiting,No Results Available,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",https://ClinicalTrials.gov/show/NCT04372602
2,3,Decitabine for Coronavirus (COVID-19) Pneumoni...,Recruiting,No Results Available,COVID-19,Drug: Decitabine|Other: Placebo Saline,"Johns Hopkins University, Baltimore, Maryland,...",https://ClinicalTrials.gov/show/NCT04482621
4,5,A Phase 2 Trial of Infliximab in Coronavirus D...,Recruiting,No Results Available,COVID-19,Drug: Infliximab,"Tufts Medical Center, Boston, Massachusetts, U...",https://ClinicalTrials.gov/show/NCT04425538
5,6,Safety & Performance Evaluation of Seraph 100 ...,Not yet recruiting,No Results Available,COVID-19,Device: Seraph 100,"Carla Kikken-Jussen, Vaals, Limburg, Netherlands",https://ClinicalTrials.gov/show/NCT04547257
6,7,Safety and Efficacy of Tramadol in COVID-19 Eg...,Not yet recruiting,No Results Available,COVID-19,Drug: Tramadol|Other: Standard care delivered ...,NaN,https://ClinicalTrials.gov/show/NCT04454307


In [5]:
# would need to parse title for phase


In [6]:
#ctrials_df[['Rank', 'Title', 'Interventions']].set_index(['Rank', 'Title'])['Interventions'].str.split('|')
ctrials_df.set_index(['Rank'], inplace=True)

ctrials_df['s1'] = ctrials_df['Interventions'].str.split('|')
display(ctrials_df.shape)
display(ctrials_df['Interventions'].__class__)
display(ctrials_df['s1'].__class__)
display(ctrials_df.__class__)
display(ctrials_df[['Interventions','s1']].head())


(3604, 8)

pandas.core.series.Series

pandas.core.series.Series

pandas.core.frame.DataFrame

,Interventions,s1
Rank,,
1,Drug: Duvelisib|Procedure: Peripheral blood dr...,"[Drug: Duvelisib, Procedure: Peripheral blood ..."
3,Drug: Decitabine|Other: Placebo Saline,"[Drug: Decitabine, Other: Placebo Saline]"
5,Drug: Infliximab,[Drug: Infliximab]
6,Device: Seraph 100,[Device: Seraph 100]
7,Drug: Tramadol|Other: Standard care delivered ...,"[Drug: Tramadol, Other: Standard care delivere..."


In [7]:
# now create one row for each array element in column s1
ctrials_df = explode(ctrials_df, 's1', preserve_index=True)
ctrials_df.reset_index(inplace=True)
display(ctrials_df.shape)
display(ctrials_df.head())


(5933, 9)

,index,Conditions,Interventions,Locations,Status,Study Results,Title,URL,s1
0,1,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",Recruiting,No Results Available,Duvelisib to Combat COVID-19,https://ClinicalTrials.gov/show/NCT04372602,Drug: Duvelisib
1,1,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",Recruiting,No Results Available,Duvelisib to Combat COVID-19,https://ClinicalTrials.gov/show/NCT04372602,Procedure: Peripheral blood draw
2,1,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",Recruiting,No Results Available,Duvelisib to Combat COVID-19,https://ClinicalTrials.gov/show/NCT04372602,Drug: Placebo
3,3,COVID-19,Drug: Decitabine|Other: Placebo Saline,"Johns Hopkins University, Baltimore, Maryland,...",Recruiting,No Results Available,Decitabine for Coronavirus (COVID-19) Pneumoni...,https://ClinicalTrials.gov/show/NCT04482621,Drug: Decitabine
4,3,COVID-19,Drug: Decitabine|Other: Placebo Saline,"Johns Hopkins University, Baltimore, Maryland,...",Recruiting,No Results Available,Decitabine for Coronavirus (COVID-19) Pneumoni...,https://ClinicalTrials.gov/show/NCT04482621,Other: Placebo Saline


In [8]:
new = ctrials_df["s1"].str.split(":", n = 1, expand = True) 
#display(new)

ctrials_df["iType"]= new[0].astype('str').str.strip()
ctrials_df["iName"]= new[1].astype('str').str.strip()


In [9]:
del new

# Dropping old Name columns 
ctrials_df.drop(columns =['Interventions', "s1"], inplace = True)


In [10]:
# show count of rows by intervention type - only some should contain small-molecule drugs
#  but the classification is not consistently accurate so use all to find matches
display(pd.DataFrame(ctrials_df.groupby(['iType'])['iType'].count()))


,iType
iType,
Behavioral,390
Biological,651
Combination Product,53
Device,286
Diagnostic Test,513
Dietary Supplement,138
Drug,2336
Genetic,16
Other,1384


In [11]:
# consider adding index/id of clinical trials input as a column of arrays

# keep only rows with intervention type of "Drug"
ctrials_drugs_df = ctrials_df[ ctrials_df['iType'] == 'Drug' ] 

# check results
display('ctrials_drugs_df shape (rows, columns)', ctrials_drugs_df.shape)
print()

# get input row IDs for each iType + iName
df2 = pd.DataFrame(ctrials_drugs_df[['iType', 'iName', 'index']].groupby(['iType', 'iName']).aggregate(lambda tdf: np.array(tdf.unique())))
df2.columns = ['inputRowIDs']

# counting number of row IDs for each iType + iName
Drug_iNames_df = pd.DataFrame(ctrials_drugs_df.groupby(['iType', 'iName'])['iType'].count())
Drug_iNames_df.columns = ['inputRowCount']
Drug_iNames_df['inputRowIDs'] = df2['inputRowIDs']
Drug_iNames_df.reset_index(inplace=True)
display("drug_iNames_df has one row per intervention name plus descriptive columns:", Drug_iNames_df.shape)
display(Drug_iNames_df[15:20])

# cleanup memory
del df2


'ctrials_drugs_df shape (rows, columns)'

(2336, 9)

'drug_iNames_df has one row per intervention name plus descriptive columns:'

(1313, 4)

,iType,iName,inputRowCount,inputRowIDs
15,Drug,1% Hydrogen Peroxide,1,212
16,Drug,1% w/v Povidone-iodide,1,2295
17,Drug,1.5-2% w/v Hydrogen Peroxide,1,2295
18,Drug,10% Povidone-iodine nasal decolonization swab ...,1,830
19,Drug,100 mg/mL Virazole,2,"[1353, 1365]"


In [12]:
# repeat processing above for ALL intervention types
# keep only rows with intervention type of "Drug"
allctrials_drugs_df = ctrials_df 

# check results
display('ctrials_drugs_df shape (rows, columns)', allctrials_drugs_df.shape)
print()

# get input row IDs for each iType + iName
df2 = pd.DataFrame(allctrials_drugs_df[['iName', 'index']].groupby(['iName']).aggregate(lambda tdf: np.array(tdf.unique())))
df2.columns = ['inputRowIDs']

# counting number of row IDs for each iType + iName
allDrug_iNames_df = pd.DataFrame(allctrials_drugs_df.groupby(['iName'])['iName'].count())
allDrug_iNames_df.columns = ['inputRowCount']
allDrug_iNames_df['inputRowIDs'] = df2['inputRowIDs']
allDrug_iNames_df.reset_index(inplace=True)
display("drug_iNames_df has one row per intervention name plus descriptive columns:", allDrug_iNames_df.shape)
display(allDrug_iNames_df[15:20])

# cleanup memory
del df2


'ctrials_drugs_df shape (rows, columns)'

(5933, 9)

'drug_iNames_df has one row per intervention name plus descriptive columns:'

(4075, 3)

,iName,inputRowCount,inputRowIDs
15,0.9% saline,3,"[248, 766, 2293]"
16,0.9% sodium chloride (normal saline),1,497
17,0.9%NaCl,1,2279
18,0.9%sodium chloride,1,669
19,1% Hydrogen Peroxide,1,212


# Load synonyms from DrugBank

In [13]:
# app.py

import json

fObj = open(DBPath+'DrugBank_CSVs/aliases.json',)
DBAlias = json.load(fObj)
fObj.close()

print(DBAlias.__class__)
display(DBAlias['DB12466'])
display(DBAlias["DB15327"])
#display(DBAlias["DB00898"]) # prednisone



<class 'dict'>


['Avigan', 'Fapilavir', 'Favilavir', 'Favipiravir']

['Abivertinib']

# match intervention Names against Drug Bank to identify DB-IDs

In [14]:
# search dict in DBAlias format to return ID if element of an array matches
def searchDBAlias(byVal):
    keysList = []
    itemsList = DBAlias.items()
    for item in itemsList:
        if byVal in item[1]:
            keysList.append(item[0])
    return keysList

In [15]:
# this could be sped up if it becomes an issue
start = time.time()
Drug_iNames_df['iDBID'] = Drug_iNames_df['iName'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)


lapse time to match:  11.452619314193726


In [16]:
Drug_iNames_df['iDBIDCount'] = Drug_iNames_df['iDBID'].astype('str').str.count("'")/2 #.str.shape()
display("Table of DrugBank matches found by intervention name", Drug_iNames_df['iDBIDCount'].value_counts())

#display(Drug_iNames_df['iDBID'].__class__)
#display(Drug_iNames_df['iDBID'].astype('str').__class__)
#display(Drug_iNames_df['iDBID'].astype('str')[250])
#display(Drug_iNames_df['iDBID'].astype('str')[400:410])


'Table of DrugBank matches found by intervention name'

0.0    1063
1.0     237
2.0      12
4.0       1
Name: iDBIDCount, dtype: int64

In [17]:
display("intervention names matching 2 Drug Bank entries", Drug_iNames_df[Drug_iNames_df['iDBIDCount']==2])


'intervention names matching 2 Drug Bank entries'

,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
250,Drug,Candesartan,1,460,"[DB00796, DB13919]",2.0
349,Drug,Diphenhydramine,1,1069,"[DB01075, DB14487]",2.0
494,Drug,Hydrocortisone,3,"[454, 1069, 1831]","[DB00741, DB14539]",2.0
677,Drug,Lopinavir-Ritonavir,2,"[2642, 3035]","[DB00503, DB01601]",2.0
771,Drug,Naltrexone,3,"[890, 1292, 2587]","[DB00620, DB00704]",2.0
939,Drug,Povidone-iodine,1,809,"[DB05382, DB06812]",2.0
944,Drug,Prednisone,4,"[381, 454, 613, 1586]","[DB00635, DB00898]",2.0
1118,Drug,TNKase,1,2930,"[DB00031, DB02325]",2.0
1199,Drug,Vitamin C,5,"[1338, 2119, 2704, 3274, 4202]","[DB00126, DB14482]",2.0
1201,Drug,Vitamin D,2,"[1885, 3586]","[DB00153, DB11094]",2.0


In [18]:
display("sample of intervention names matching 1 Drug Bank entries", Drug_iNames_df[Drug_iNames_df['iDBIDCount']==1].head())


'sample of intervention names matching 1 Drug Bank entries'

,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
42,Drug,AMY-101,1,1891,[DB14803],1.0
49,Drug,AT-001,1,308,[DB15121],1.0
61,Drug,AZD1222,1,14,[DB15656],1.0
66,Drug,Abatacept,2,"[309, 583]",[DB01281],1.0
69,Drug,Abivertinib,1,1352,[DB15327],1.0


In [19]:
# repeat matching - but for ALL intervention types to see if drugs in drug bank were misclassified
start = time.time()
allDrug_iNames_df['iDBID'] = allDrug_iNames_df['iName'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)



lapse time to match:  33.486955881118774


In [20]:

allDrug_iNames_df['iDBIDCount'] = allDrug_iNames_df['iDBID'].astype('str').str.count("'")/2 
display(allDrug_iNames_df['iDBIDCount'].value_counts())



0.0     3796
1.0      264
2.0       12
16.0       1
7.0        1
4.0        1
Name: iDBIDCount, dtype: int64

including non-drug intervention types increases matches to Drug Bank 
so, ignore intervention type and move forward with ALL records 

# create exception lists for review 
## to inform text cleanup below

In [21]:
allDrug_iNames_df[allDrug_iNames_df['iDBIDCount'] > 0].head()

,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
83,AMY-101,1,1891,[DB14803],1.0
108,AT-001,1,308,[DB15121],1.0
124,AZD1222,3,"[14, 19, 231]",[DB15656],1.0
129,Abatacept,3,"[309, 583, 607]",[DB01281],1.0
133,Abivertinib,1,1352,[DB15327],1.0


# Write out ClinicalTrialsTruth.tsv file

DBID, iType, iName (most common? or list ?), inputRows, status, study_results, (parse phase?)

In [22]:
ProposedTruth_df = allDrug_iNames_df[ allDrug_iNames_df['iDBIDCount'] > 0 ].copy()
display(ProposedTruth_df.head())
display(ProposedTruth_df.iDBID.__class__)
display(ProposedTruth_df.iDBID.shape)
display(ProposedTruth_df.iDBID.iloc[1])
display(ProposedTruth_df.iDBID.iloc[1].__class__)

#expand out to one row per intervention-Drubank ID 
s = ProposedTruth_df.apply(lambda x: pd.Series(x['iDBID']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'iDBID'
#display(s)

ProposedTruth_df = ProposedTruth_df.drop('iDBID', axis=1).join(s)
display(ProposedTruth_df[10:30])

,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
83,AMY-101,1,1891,[DB14803],1.0
108,AT-001,1,308,[DB15121],1.0
124,AZD1222,3,"[14, 19, 231]",[DB15656],1.0
129,Abatacept,3,"[309, 583, 607]",[DB01281],1.0
133,Abivertinib,1,1352,[DB15327],1.0


pandas.core.series.Series

(279,)

['DB15121']

list

,iName,inputRowCount,inputRowIDs,iDBIDCount,iDBID
184,Alisporivir,1,2226,1.0,DB12139
190,Almitrine,1,2180,1.0,DB01430
194,Alteplase,1,3752,1.0,DB00009
196,Alvelestat,1,1162,1.0,DB11863
197,Ambrisentan,1,804,1.0,DB06403
198,Amiodarone,1,340,1.0,DB01118
199,Amlodipine,1,3746,1.0,DB00381
201,Ampion,2,"[453, 1288]",1.0,DB14940
203,Anakinra,10,"[271, 1841, 2149, 2263, 2330, 2690, 2974, 3274...",1.0,DB00026
216,Angiotensin 1-7,2,"[705, 2964]",1.0,DB11720


In [23]:
ProposedTruth_df.to_csv("ProposedTruth.csv")


# Concept for Future Development

## apply manual override file (for when just typing in the change is easier)

## NOTE: any manual overrides should be entered into a .csv file and treated as data


In [24]:
# add an additional file to INSERT complicated records that contain more than one drug
# 541	Drug	Hydroxychloroquine, Clindamycin, Primaquine - ...


In [ ]:
# read exception override file
iName_Overrides_df = pd.read_csv(CTPath + 'ctrials_iname_overrides.csv', delimiter='\t')
display(iName_Overrides_df.head())


In [28]:
# apply exception override file
Drug_iNamesNew_df = pd.merge(Drug_iNames_df, iName_Overrides_df, how='left', on='iName', indicator=True)
display(Drug_iNamesNew_df['_merge'].value_counts())

Drug_iNamesNew_df.rename(columns = {'iName':'iNameOld'}, inplace = True)
Drug_iNamesNew_df = Drug_iNamesNew_df[Drug_iNamesNew_df._merge=='both']
display(Drug_iNamesNew_df.head())


left_only     1312
both             1
right_only       0
Name: _merge, dtype: int64

,iType,iNameOld,inputRowCount,inputRowIDs,iDBID,iDBIDCount,iNameNew,_merge
92,Drug,Alteplase 50 MG [Activase],1,3155,[],0.0,Alteplase,both


### now match cleaned text into drug-bank

In [29]:
# repeat matching - but for ALL intervention types to see if drugs in drug bank were misclassified
start = time.time()
Drug_iNamesNew_df['iDBID'] = Drug_iNamesNew_df['iNameNew'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)



lapse time to match:  0.1085197925567627


In [30]:

Drug_iNamesNew_df['iDBIDCount'] = Drug_iNamesNew_df['iDBID'].astype('str').str.count("'")/2 
display(Drug_iNamesNew_df['iDBIDCount'].value_counts())



1.0    1
Name: iDBIDCount, dtype: int64

In [31]:
display(Drug_iNamesNew_df.head())

,iType,iNameOld,inputRowCount,inputRowIDs,iDBID,iDBIDCount,iNameNew,_merge
92,Drug,Alteplase 50 MG [Activase],1,3155,[DB00009],1.0,Alteplase,both
